In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np

In [2]:
def get_data(nome_arquivo,shuffle):
    ds = pd.read_csv(nome_arquivo,encoding="utf-8")
    if shuffle:
        ds = ds.sample(frac=1)
    ds['texto'] = ds['texto'].apply(str)
    return ds

In [3]:
df = get_data("train.csv",1)

In [4]:
df.shape

(820, 2)

In [5]:
df.head()

,valor,texto
654,0,Esta monografia apresenta o estudo e projeto d...
378,1,A osteoporose é uma doença do esqueleto caract...
120,1,Esse trabalho apresenta a concepção e implemen...
172,1,Os parâmetros S são uma importante ferramenta ...
322,1,O objetivo deste trabalho é o desenvolvimento ...


In [6]:
df.texto

654    Esta monografia apresenta o estudo e projeto d...
378    A osteoporose é uma doença do esqueleto caract...
120    Esse trabalho apresenta a concepção e implemen...
172    Os parâmetros S são uma importante ferramenta ...
322    O objetivo deste trabalho é o desenvolvimento ...
                             ...                        
645    Nesta monografia, analisou-se as consequências...
709    Nos últimos anos a Geração Distribuída (GD) te...
741    Nos últimos anos, notou-se um grande crescimen...
414    Este projeto tem por objetivo o desenvolviment...
195    Os campos da robótica e biologia apresentam di...
Name: texto, Length: 820, dtype: object

In [7]:
print((df.valor==1).sum())#eletronica
print((df.valor==0).sum())#eletrica

410
410


In [8]:
import re
import string
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("portuguese"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",text)

def remove_punct(text):
    translator = str.maketrans("","",'!""#$%&\'()*+,./:;<=>?@[\\]^_`{|}~º')
    translator = str.maketrans("","",'!""#$%&\'()*+,./:;<=>?@[\\]^_`{|}~º')
    return text.translate(translator)

def remove_numbers(text):
    result = ''.join([i for i in text if not i.isdigit()])
    return result

def remove_hifen(text):
    translator = str.maketrans('-',' ')
    return text.translate(translator)

In [9]:
pattern = re.compile(r"https?//(\S+|www)\.\S+")
def pat(df_t):
    for t in df_t.texto:
        matches = pattern.findall(t)
        for match in  matches:
            print(t)
            print(match)
            print(pattern.sub(r"",t))        
        if len(matches)> 0:
            break
pat(df)            

In [10]:
def make_test(df_t):
    df_t["texto"] = df_t.texto.map(remove_URL)
    df_t["texto"] = df_t.texto.map(remove_punct)
    df_t["texto"] = df_t.texto.map(remove_hifen)
    df_t["texto"] = df_t.texto.map(remove_numbers)
    df_t["texto"] = df_t.texto.map(remove_stopwords)

make_test(df)

In [11]:
from collections import Counter

def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

counter = counter_word(df.texto)

In [12]:
len(counter)

11773

In [13]:
counter.most_common(5)

[('sistema', 1066),
 ('trabalho', 940),
 ('energia', 686),
 ('projeto', 466),
 ('elétrica', 404)]

In [14]:
num_unique_words = len(counter)

In [15]:
def data_split(df,size):
    train_size = int(df.shape[0]*size)
    train_df = df[:train_size]
    val_df = df[train_size:]
    return train_df, val_df

train_df, val_df = data_split(df,0.8)

In [16]:
print(len(train_df))
print(len(val_df))

656
164


In [17]:
def data_to_numpy(df):    
    train_sentences = train_df.texto.to_numpy()
    train_labels = train_df.valor.to_numpy()
    val_sentences = val_df.texto.to_numpy()
    val_labels = val_df.valor.to_numpy()
    return train_sentences, train_labels, val_sentences, val_labels

train_sentences, train_labels, val_sentences, val_labels = data_to_numpy(df)

In [18]:
train_sentences.shape, val_sentences.shape

((656,), (164,))

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = num_unique_words,oov_token="<OOV>")

def tokenization(df):      
    tokenizer.fit_on_texts(train_sentences)
    word_index = tokenizer.word_index
    train_sequences = tokenizer.texts_to_sequences(train_sentences)
    val_sequences = tokenizer.texts_to_sequences(val_sentences)
    return train_sequences, val_sequences, word_index

train_sequences, val_sequences, word_index = tokenization(df)

In [20]:
#word_index = tokenizer.word_index

In [21]:
#train_sequences = tokenizer.texts_to_sequences(train_sentences)
#val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 600

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding = "post",truncating = "post")
val_padded = pad_sequences(val_sequences, maxlen = max_length, padding = "post", truncating = "post")
train_padded.shape, val_padded.shape

((656, 600), (164, 600))

In [23]:
reverse_word_index = dict([(idx,word) for (word, idx) in word_index.items()])

In [24]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [25]:
decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)

[3, 211, 233, 13, 709, 938, 2662, 200, 561, 63, 11, 2663, 21, 2664, 16, 1080, 4, 2265, 2665, 184, 200, 561, 534, 876, 125, 167, 45, 18, 3208, 132, 159, 4071, 1171, 5741, 1080, 4, 2665, 313, 602, 160, 3209, 1080, 4, 2265, 2665, 16, 4072, 30, 515, 325, 4073, 1080, 30, 1723, 599, 79, 877, 4, 2665, 51, 215, 3196, 115, 1080, 6, 132, 200, 561, 534, 876, 151, 369, 1172, 1282, 383, 1080, 2666, 31, 1014, 263, 3210, 633, 53, 599, 247]
trabalho visa realização estudo determinação diferenças introduzidas planejamento energético operação sistemas hidrotérmicos potência variar forma representação energia natural afluente atualmente planejamento energético médio prazo brasil realizado utilização software newave utilizada duas mil séries sintéticas representação energia afluente modelagem aqui proposta sugere representação energia natural afluente forma determinística meio média longo termo representação meio múltiplos cenários obtidos histórico energia afluente testes realizados indicarão melhor repr

In [26]:
from tensorflow.keras import layers

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length = max_length))

#model.add(layers.LSTM(256,dropout = 0.1))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(24, activation = "relu"))
model.add(layers.Dense(2, activation = "softmax"))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 600, 32)           376736    
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 50        
Total params: 400,618
Trainable params: 400,618
Non-trainable params: 0
__________________________________________________

In [27]:
#loss = keras.losses.BinaryCrossentropy(from_logits=False)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer = optim, metrics = metrics)

In [28]:
import time
start = time.perf_counter()
model.fit(train_padded,train_labels, epochs = 20, validation_data=(val_padded,val_labels), verbose=2)
finish = time.perf_counter()
print(f'\nFinished in {round(finish-start, 2)} second(s)')

Epoch 1/20
21/21 - 1s - loss: 0.6938 - accuracy: 0.4878 - val_loss: 0.6932 - val_accuracy: 0.4817
Epoch 2/20
21/21 - 0s - loss: 0.6931 - accuracy: 0.5107 - val_loss: 0.6931 - val_accuracy: 0.4695
Epoch 3/20
21/21 - 0s - loss: 0.6926 - accuracy: 0.5076 - val_loss: 0.6925 - val_accuracy: 0.5854
Epoch 4/20
21/21 - 0s - loss: 0.6905 - accuracy: 0.6082 - val_loss: 0.6898 - val_accuracy: 0.7317
Epoch 5/20
21/21 - 0s - loss: 0.6808 - accuracy: 0.6814 - val_loss: 0.6754 - val_accuracy: 0.7805
Epoch 6/20
21/21 - 0s - loss: 0.6162 - accuracy: 0.8308 - val_loss: 0.6163 - val_accuracy: 0.7195
Epoch 7/20
21/21 - 0s - loss: 0.4310 - accuracy: 0.8674 - val_loss: 0.5328 - val_accuracy: 0.7317
Epoch 8/20
21/21 - 0s - loss: 0.2287 - accuracy: 0.9345 - val_loss: 0.5947 - val_accuracy: 0.7256
Epoch 9/20
21/21 - 0s - loss: 0.1310 - accuracy: 0.9466 - val_loss: 0.6264 - val_accuracy: 0.7378
Epoch 10/20
21/21 - 0s - loss: 0.0748 - accuracy: 0.9817 - val_loss: 0.6212 - val_accuracy: 0.7805
Epoch 11/20
21/21 -

In [29]:
predictions = model.predict(val_padded)
print([np.argmax(element) for element in predictions])

#predictions = [1 if p > 0.5 else 0 for p in predictions]

[0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]


In [30]:
index = 7
#print(val_sentences[index])
#print(val_padded[index],'\n')
print("Label: ",val_labels[index])
print("Resultado: ",predictions[index],'\n')
print(val_labels,'\n')
print(predictions)

Label:  1
Resultado:  [0.11804704 0.88195294] 

[0 1 1 1 0 0 0 1 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 1
 1 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 1 1 1 1
 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0
 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0
 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 1] 

[[9.9993181e-01 6.8208152e-05]
 [7.8590488e-01 2.1409515e-01]
 [3.7417797e-04 9.9962580e-01]
 [1.9376443e-04 9.9980623e-01]
 [9.9873263e-01 1.2673312e-03]
 [8.6250831e-04 9.9913752e-01]
 [9.9911386e-01 8.8607409e-04]
 [1.1804704e-01 8.8195294e-01]
 [5.6490165e-01 4.3509838e-01]
 [5.2320473e-03 9.9476802e-01]
 [9.6204740e-01 3.7952565e-02]
 [9.9854857e-01 1.4513747e-03]
 [9.9814844e-01 1.8516252e-03]
 [1.5023846e-05 9.9998498e-01]
 [5.3517397e-02 9.4648260e-01]
 [9.9731195e-01 2.6880482e-03]
 [9.9121958e-01 8.7803658e-03]
 [2.5971059e-03 9.9740285e-01]
 [1.2489156e-01 8.7510848e-01]
 [9.9982315e-01 1.7691849e-

In [31]:
df_t = get_data("eval.csv",0)

In [32]:
df_t.shape

(8, 2)

In [33]:
df_t.head()

,valor,texto
0,1,Este projeto visa uma nova abordagem para um s...
1,1,"Nesta monografia, é apresentada a proposta de ..."
2,1,O objetivo da monografia foi estudar como as t...
3,1,As células de carga são instrumentos versáteis...
4,0,"Na atualidade, a produção termelétrica partici..."


In [34]:
pat(df_t)
make_test(df_t)

In [35]:
df_t.texto

0    projeto visa nova abordagem sistema vigilância...
1    nesta monografia apresentada proposta equipame...
2    objetivo monografia estudar técnicas controle ...
3    células carga instrumentos versáteis usados mo...
4    atualidade produção termelétrica participa mar...
5    neste trabalho apresentado projeto instalação ...
6    trabalho intitulado “estimação preço demanda e...
7    trabalho apresenta estudo sobre impactos causa...
Name: texto, dtype: object

In [36]:
test_sentences = df_t.texto.to_numpy()
test_labels = df_t.valor.to_numpy()

In [37]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [38]:
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding = "post",truncating = "post")

In [39]:
predictions_t = model.predict(test_padded)
print(predictions_t)
#predictions_t = [1 if p > 0.5 else 0 for p in predictions_t]


[[2.5721414e-03 9.9742788e-01]
 [7.1325824e-02 9.2867416e-01]
 [9.8352700e-01 1.6473042e-02]
 [1.0946973e-02 9.8905307e-01]
 [9.9972707e-01 2.7290345e-04]
 [9.9971825e-01 2.8174731e-04]
 [9.3418986e-01 6.5810189e-02]
 [9.9984109e-01 1.5886356e-04]]


In [40]:
index = 6
print(test_sentences[2])#[index])

print(test_labels)#[index])
#print(predictions_t)#[index])
predictions_t = [np.argmax(element) for element in predictions_t]
print(predictions_t)
def precision_t():
    counter = len(test_labels)
    list_c = [i for i,j in zip(predictions_t,test_labels) if i == j]
    return len(list_c)/counter*100

print(precision_t(),"%")

objetivo monografia estudar técnicas controle linear utilizadas determinar controladores largamente conhecidos avr pss impactam margem estabilidade transitória sistema máquina versus barramento infinito omib one machine versus infinite bus adota modelo eixo gerador síncrono pólos salientes seguida estudam modelos avr pss apresentados norma ieee standard ieee standard adotando modelo pssa single input daquela norma equações diferenciais sistema numericamente integradas sistema então simulado sob perturbação resposta dinâmica analisada averiguadas ocorrências bifurcações hopf sistema função parâmetros controladores através parametrização autovalores análise comportamento local sistema equilíbrio traçam diagramas bifurcação sistema baseando teoria regiões estabilidade sistemas dinâmicos lineares simulações utilizadas desenvolver método força bruta mfb estimar região estabilidade sistema malha aberta controlado avr controlado avr pss saturadores excitação traçam regiões estabilidade varian

In [41]:
def prepare(teste):
    teste = remove_URL(teste)
    teste = remove_punct(teste)
    teste = remove_hifen(teste)
    teste = remove_stopwords(teste)    
    return teste

def predict(teste):
    predictions = model.predict(np.array(teste)) 
    p1 = [np.argmax(element) for element in predictions]
    if p1[0]:
        print("Disciplina: Eletronica")
    else:
        print("Disciplina: Elétrica")
    return predictions,p1



In [42]:
teste_1 = "O objetivo deste trabalho é propor um algoritmo para realizar a identificação de padrões na vocalização suína, visando determinar o nível do bem-estar do animal. Tal análise foi proposta uma vez que o bem-estar animal é um assunto cada vez mais abordado no mundo todo, principalmente quando os animais são criados para o abate. Dessa forma, a criação de um método em que haja o mínimo de contato com os animais se faz importante, evitando que tal contato altere o comportamento do animal e, conseqüentemente, o resultado da análise de seu bem-estar. Por essas características, foi proposto um método de análise dos sons emitidos pelos suínos com base na utilização de uma Rede Neural Artificial do tipo Radial Basis Function, a qual possui como elementos de treinamento e operação um conjunto de características extraídas através da Transformada Discreta Wavelet de sinais sonoros pré-gravados. As características obtidas dos sinais foram as energias das bandas críticas relativas à Escala Bark e a diferença entre as energias das bandas adjacentes, além dimensão fractal do sinal. Através desse método foram analisados dois tipos de sinais sonoros: a vocalização de leitões saudáveis e de leitões acometidos por uma doença chamada Artrite Traumática; e a vocalização de suínos adultos em situações de conforto e desconforto. Os resultados demonstram que a análise proposta atingiu bons patamares de acerto na determinação do bem-estar do animal"
#eletronica

In [43]:
teste_1 = tokenizer.texts_to_sequences([prepare(teste_1)])

In [44]:
predict(teste_1)

Disciplina: Eletronica


(array([[0.00363937, 0.9963606 ]], dtype=float32), [1])

In [45]:
teste_2 = "Neste projeto de formatura seria dada continuidade ao trabalho de iniciação científica, que consistiu na análise de métodos de coordenação de robôs móveis, ou seja métodos de controle de um conjunto de robôs para que esses assumam uma determinada formação com direção e distância entre eles pré-definidas. Sendo estudada primeiramente a cinemática do robô móvel e logo em seguida foi feita a análise de uma formação simples de líder-seguidor, considerando também a alternância da liderança entre os robôs. A partir desse ponto seria feita a implementação da simulação do algoritmo de controle de 6 robôs móveis em formação utilizando controle por grafos e sistemas lineares sujeitos a saltos markovianos"
#eletronica

In [46]:
teste_2 = tokenizer.texts_to_sequences([prepare(teste_2)])

In [47]:
predict(teste_2)

Disciplina: Elétrica


(array([[9.9943095e-01, 5.6906685e-04]], dtype=float32), [0])